In [2]:
import easyocr
import cv2
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import os
import pytesseract
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [3]:
pd.set_option('display.max_colwidth', None)

## Import CSV File

In [4]:
label = pd.read_csv('license_plate_dataset\lpr.csv')
label.head()

,images,labels
0,1.jpg,RZ0047
1,10.jpg,V95246
2,100.jpg,6258TU
3,10000.jpg,B88082
4,10001.jpg,7065UK


##  Define Image and Label Path

In [5]:
image_path = 'license_plate_dataset\cropped_lps\cropped_lps'

label_path = 'license_plate_dataset\lpr.csv'

label_df = pd.read_csv(label_path, header=None, names=['image_filename','label'])

label_df['image_path'] = label_df['image_filename'].apply(lambda x: os.path.join(image_path, x))
label_df = label_df.drop(label_df.index[0])
label_df = label_df.reset_index(drop=True)
label_df.head()

,image_filename,label,image_path
0,1.jpg,RZ0047,license_plate_dataset\cropped_lps\cropped_lps\1.jpg
1,10.jpg,V95246,license_plate_dataset\cropped_lps\cropped_lps\10.jpg
2,100.jpg,6258TU,license_plate_dataset\cropped_lps\cropped_lps\100.jpg
3,10000.jpg,B88082,license_plate_dataset\cropped_lps\cropped_lps\10000.jpg
4,10001.jpg,7065UK,license_plate_dataset\cropped_lps\cropped_lps\10001.jpg


## Tesseract OCR

In [25]:
# Set Tesseract executable path
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

def perform_ocr_with_tesseract(image_path):
    
    img = Image.open(image_path)
   
    text = pytesseract.image_to_string(img, lang='eng')
    return text.strip()

In [26]:
def process_and_evaluate(df):
    labels = []
    predictions = []

    for index, row in df.iterrows():
        image_path = row['image_path']  
        label = row['label']  

        tesseract_ocr_result = perform_ocr_with_tesseract(image_path)

        labels.append(label)
        predictions.append(tesseract_ocr_result)

        print(f'Image: {image_path}')
        print(f'Label: {label}')
        print(f'Tesseract OCR Result: {tesseract_ocr_result}')
        print('---')

    return labels, predictions

In [ ]:
labels, predictions = process_and_evaluate(label_df)

accuracy = accuracy_score(labels, predictions)

print(f'Tesseract Accuracy: {accuracy}')

## Sharpen Images

In [10]:
# Function to sharpen the image
def sharpen_image(image):
    # Create the sharpening kernel
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    # Sharpen the image
    sharpened_image = cv2.filter2D(image, -1, kernel)
    return sharpened_image


image_paths = label_df['image_path'].tolist()

sharpened_folder = 'sharpened_images'
os.makedirs(sharpened_folder, exist_ok=True)

# Iterate through the image paths
for i, image_path in enumerate(image_paths):
    # Load the image
    image = cv2.imread(image_path)
    
    # Sharpen the image
    sharpened_image = sharpen_image(image)
    
    # Save the sharpened image
    sharpened_image_path = os.path.join(sharpened_folder, f'sharpened_{i+1}.jpg')
    cv2.imwrite(sharpened_image_path, sharpened_image)

    label_df.at[i, 'sharpened_image_path'] = sharpened_image_path


print(f"Finished saving sharpened image saved in {sharpened_folder}")

Finished saving sharpened image saved in sharpened_images


In [11]:
label_df.head()

,image_filename,label,image_path,sharpened_image_path
0,1.jpg,RZ0047,license_plate_dataset\cropped_lps\cropped_lps\1.jpg,sharpened_images\sharpened_1.jpg
1,10.jpg,V95246,license_plate_dataset\cropped_lps\cropped_lps\10.jpg,sharpened_images\sharpened_2.jpg
2,100.jpg,6258TU,license_plate_dataset\cropped_lps\cropped_lps\100.jpg,sharpened_images\sharpened_3.jpg
3,10000.jpg,B88082,license_plate_dataset\cropped_lps\cropped_lps\10000.jpg,sharpened_images\sharpened_4.jpg
4,10001.jpg,7065UK,license_plate_dataset\cropped_lps\cropped_lps\10001.jpg,sharpened_images\sharpened_5.jpg


In [18]:
sharpened_image_path = 'sharpened_images'

## OCR Using Tesseract with Sharpened Images

In [22]:
# Set Tesseract executable path
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

def perform_ocr_with_tesseract(sharpened_image_path):
    
    img = Image.open(sharpened_image_path)
    
    text = pytesseract.image_to_string(img, lang='eng')
    return text.strip()

In [23]:
def process_and_evaluate(df):
    labels = []
    predictions = []

    for index, row in df.iterrows():
        sharpened_image_path = row['sharpened_image_path']  
        label = row['label']  

        tesseract_ocr_result = perform_ocr_with_tesseract(sharpened_image_path)

        labels.append(label)
        predictions.append(tesseract_ocr_result)

        print(f'Image: {sharpened_image_path}')
        print(f'Label: {label}')
        print(f'Tesseract OCR Result: {tesseract_ocr_result}')
        print('---')

    return labels, predictions

In [ ]:
labels, predictions = process_and_evaluate(label_df)

accuracy = accuracy_score(labels, predictions)

print(f'Tesseract Accuracy: {accuracy}')

## OCR Using EasyOCR

In [6]:
import easyocr

reader = easyocr.Reader(['en'])

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [7]:
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    
    if img is None:
        print(f"Error: Could not load image {image_path}.")
        return None
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Image Binarization
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV)
    
    # Denoised the binarized image
    denoised = cv2.fastNlMeansDenoising(binary, None, 30, 7, 21)
    
    return denoised
    
def perform_ocr(image_path):
    preprocessed_img = preprocess_image(image_path)
    
    if preprocessed_img is None:
        return ""
    
    result = reader.readtext(preprocessed_img, detail=0, paragraph=True)
    text = ' '.join(result)
    return text.strip()

In [12]:
def process_and_evaluate(df):
    labels = []
    predictions = []

    for index, row in df.iterrows():
        image_path = row['image_path']  
        label = row['label']  

        ocr_result = perform_ocr(image_path)

        labels.append(label)
        predictions.append(ocr_result)

        print(f'Image: {image_path}')
        print(f'Label: {label}')
        print(f'EasyOCR Result: {ocr_result}')
        print('---')

    return labels, predictions

In [ ]:
labels, predictions = process_and_evaluate(label_df)

# Calculate accuracy
accuracy = accuracy_score(labels, predictions)

print(f'EasyOCR Accuracy: {accuracy}')

## Summary
### Tesseract with Non-Sharpened Images
Tesseract Accuracy: 0.008

Run Time: 49m 17.6s
### Tesseract with Sharpened Images
Tesseract Accuracy: 0.00165

Run Time: 48m 24.2s
### EasyOCR with Binarization and Denoised Images
OCR Accuracy: 0.05295

Run Time: 67m 21.3s
